# Import Libraries

In [217]:
# libraries that deal with grabbing from the .env file
from dotenv import load_dotenv
import os

# make url requests
import requests

# make DFs
import pandas as pd

# show images in notebook
from IPython.display import display, Image

# Setup stuff

In [218]:
# Remove the cached environment variable (if any)
os.environ.pop('riot_api_key', None)

# Reload the .env file
load_dotenv()

# Fetch the API key
api_key = os.environ.get('riot_api_key')

# Defining Functions

## Get Puuid

In [219]:
def get_puuid(summonerId=None, gameName=None, tagLine=None, region='americas'):
    """Gets the puuid from a summonerId or Riot_id and riot_tag
    
    Args:
        summonerId (str, optional): Summoner ID. Defaults to None.
        gameName (str, optional): Riot ID. Defaults to None.
        tagLine (str, optional): Riot Tag. Defaults to None.
        region (str, optional): Region. Defaults to 'americas'.
        
    Returns:
        str: puuid
    """
# using summonerID 
    if summonerId is not None:
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = 'lol/summoner/v4/summoners/'
        print(root_url+endpoint+summonerId+'?api_key='+api_key)
        response = requests.get(root_url+endpoint+summonerId+'?api_key='+api_key)

        return response.json()['puuid']
    else:
        #using gamename + tagline for example HGILLIS + NA1
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = f'riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}'

        response = requests.get(root_url+endpoint+'?api_key='+api_key)

        return response.json()['puuid']
    


## Get match history (list of game ids)

In [220]:
def get_match_history(region=None, puuid=None, start=0, count=20):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/by-puuid/{puuid}/ids'
    query_params = f'?start={start}&count={count}' #changing starting index (start) and number of matches retrieved (count)

    response = requests.get(root_url + endpoint + query_params + '&api_key=' + api_key)
    
    return response.json()

## get info for single game

In [221]:
def get_match_data_from_id(region = None, matchId = None):

    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/{matchId}'


    response = requests.get(root_url + endpoint + '?api_key=' + api_key)

    return response.json()

## Process match history

In [222]:


def process_match_json(match_json, puuid):
    ## Architecture
    # left side is my name, right side is riots name
    metadata = match_json['metadata']
    info = match_json['info']
    game_mode = info['gameMode']
    players = info['participants']
    participants = metadata['participants']
    teams = info['teams']
    player = players[participants.index(puuid)]
    perks = player['perks']
    stats = perks ['statPerks']
    styles = perks['styles']

    primary = styles[0]
    secondary = styles[1]

    match_id = metadata['matchId']


    game_creation = info['gameCreation']
    game_duration = info['gameDuration']
    game_start_timestamp = info['gameStartTimestamp']
    game_end_timestamp = info['gameEndTimestamp']
    patch = info['gameVersion']

    riot_id = player['riotIdGameName']
    riot_tag = player['riotIdTagline']
    summoner_id = player['summonerId']
    summoner_name = player['summonerName']

    win = player['win']

    champ_id = player['championId']
    champ_transform = player['championTransform']
    champ_level = player['champLevel']

    gold_earned = player['goldEarned']
    neutral_minions_killed = player['neutralMinionsKilled']
    total_minions_killed = player['totalMinionsKilled']

    kills = player['kills']
    deaths = player['deaths']
    assists = player['assists']
    first_blood = player['firstBloodKill']

    total_damage_dealt= player['totalDamageDealtToChampions']
    total_damage_shielded = player['totalDamageShieldedOnTeammates']
    total_damage_taken = player['totalDamageTaken']
    total_damage_healed = player['totalHealsOnTeammates']
    total_minions_killed = player['totalMinionsKilled']
    total_time_cc_dealt = player['totalTimeCCDealt']

    early_surrender = player['gameEndedInEarlySurrender']
    surrender = player['gameEndedInSurrender']

    item0 = player['item0']
    item1 = player['item1']
    item2 = player['item2']
    item3 = player['item3']
    item4 = player['item4']
    item5 = player['item5']
    item6 = player['item6']

    summoner_1_id = player['summoner1Id']
    summoner_2_id = player['summoner2Id']


    wards_placed = player['wardsPlaced']
    vision_score = player['visionScore']







    defense = stats['defense']
    flex = stats['flex']
    offense = stats['offense']


    primary_style = primary['style']
    secondary_style = primary['style']

    primary_keystone = primary['selections'][0]['perk']
    primary_perk_1 = primary['selections'][1]['perk']
    primary_perk_2 = primary['selections'][2]['perk']
    primary_perk_3 = primary['selections'][3]['perk']

    secondary_perk_1 = secondary['selections'][0]['perk']
    secondary_perk_2 = secondary['selections'][1]['perk']


    objectives_stolen = player['objectivesStolen']
    objectives_stolen_assists = player['objectivesStolenAssists']

    ##what is this?
    detector_wars_placed = player['detectorWardsPlaced']
    vision_wards_bought = player['visionWardsBoughtInGame']
    role = player['role']

    damage_dealt_to_buildings = player['damageDealtToBuildings']
    damage_dealt_to_objectives = player['damageDealtToObjectives']
    damage_dealt_to_turrets = player['damageDealtToTurrets']
    game_ended_in_surrender = player['gameEndedInSurrender']
    gold_earned = player['goldEarned']
    individual_position = player['individualPosition']
    inhibitor_takedowns = player['inhibitorTakedowns']
    largest_killing_spree = player['largestKillingSpree']
    largest_multi_kill = player['largestMultiKill']
    total_damage_shielded_on_teammates = player['totalDamageShieldedOnTeammates']
    total_heals_on_teammates = player['totalHealsOnTeammates']
    total_minions_killed = player['totalMinionsKilled']
    vision_score = player['visionScore']


    matchDF = pd.DataFrame({
        'match_id': [match_id],
        'game_mode': [game_mode],
        'participants': [participants],
        'game_creation': [game_creation],
        'game_duration': [game_duration],
        'game_start_timestamp': [game_start_timestamp],
        'game_end_timestamp': [game_end_timestamp],
        'patch': [patch],
        'puuid': [puuid],
        'riot_id': [riot_id],
        'riot_tag': [riot_tag],
        'summoner_id': [summoner_id],
        'summoner_name': [summoner_name],
        'win': [win],
        'champion': [champ_id],
        'champion_transform': [champ_transform],
        'champion_level': [champ_level],
        'kills': [kills],
        'deaths': [deaths],
        'assists': [assists],
        'summoner1_id': [summoner_1_id],
        'summoner2_id': [summoner_2_id],
        'gold_earned': [gold_earned],
        'total_minions_killed': [total_minions_killed],
        'total_neutral_minions_killed': [neutral_minions_killed],
        'early_surrender': [early_surrender],
        'surrender': [surrender],
        'first_blood': [first_blood],
        'objectives_stolen': [objectives_stolen],
        'objectives_stolen_assists': [objectives_stolen_assists],
        'total_damage_dealt_champions': [total_damage_dealt],
        'total_damage_taken': [total_damage_taken],
        'total_damage_shielded_teammates': [total_damage_shielded],
        'total_time_crowd_controlled': [total_time_cc_dealt],
        'vision_score': [vision_score],
        'wards_placed': [wards_placed],
        'item0': [item0],
        'item1': [item1],
        'item2': [item2],
        'item3': [item3],
        'item4': [item4],
        'item5': [item5],
        'item6': [item6],
        'perk_keystone': [primary_keystone],
        'perk_primary_row_1': [primary_perk_1],
        'perk_primary_row_2': [primary_perk_2],
        'perk_primary_row_3': [primary_perk_3],
        'perk_secondary_row_1': [secondary_perk_1],
        'perk_secondary_row_2': [secondary_perk_2],
        'perk_primary_style': [primary_style],
        'perk_secondary_style': [secondary_style],
        'perk_shard_defense': [defense],
        'perk_shard_flex': [flex],
        'perk_shard_offense': [offense],
        'damage_dealt_to_buildings': [damage_dealt_to_buildings],
        'damage_dealt_to_objectives': [damage_dealt_to_objectives],
        'damage_dealt_to_turrets': [damage_dealt_to_turrets],
        'game_ended_in_surrender': [game_ended_in_surrender],
        'individual_position': [individual_position],
        'inhibitor_takedowns': [inhibitor_takedowns],
        'largest_killing_spree': [largest_killing_spree],
        'largest_multi_kill': [largest_multi_kill],
        'total_damage_shielded_on_teammates': [total_damage_shielded_on_teammates],
        'total_heals_on_teammates': [total_heals_on_teammates],
        'total_minions_killed': [total_minions_killed],
        'vision_score': [vision_score],
    })


    return matchDF


## go from json to english

In [223]:

def json_extract(obj, key):
    arr = []
    
    def extract(obj, arr, key):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == key:
                    arr.append(v)
                elif isinstance(v, (dict, list)):
                    extract(v, arr, key)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values


# Community Dragon

In [224]:
# keys to go from numbers to actual names

perk = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perks.json'
perk_styles = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perkstyles.json'
champ = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champion-summary.json'
item = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/items.json'

perk_json = requests.get(perk).json()
perk_styles_json = requests.get(perk_styles).json()
champ_json = requests.get(champ).json()
item_json = requests.get(item).json()

In [225]:
# code to replace all of the perk numbers with the actual perks

perk_ids = json_extract(perk_json, 'id')
perk_names = json_extract(perk_json, 'name')

perk_dict = dict(map(lambda i, j : (int(i),j), perk_ids, perk_names))

perk_styles_ids = json_extract(perk_styles_json, 'id')
perk_styles_names = json_extract(perk_styles_json, 'name')

perk_styles_dict = dict(map(lambda i, j : (int(i),j), perk_styles_ids, perk_styles_names))

In [226]:
# code to replace all of the champion numbers with the actual champions
champ_ids = json_extract(champ_json, 'id')
champ_names = json_extract(champ_json, 'name')

# Assuming champ_json is a list of dictionaries with each dictionary containing 'id' and 'name'
champ_dict = {int(champ['id']): champ['name'] for champ in champ_json}



In [227]:
# code to replace all of the item numbers with the actual items
item_ids = json_extract(item_json, 'id')
item_names = json_extract(item_json, 'name')

# Assuming champ_json is a list of dictionaries with each dictionary containing 'id' and 'name'
item_dict = {int(item['id']): item['name'] for item in item_json}


# Get player names

In [228]:
# input username, region code here, asking for user input

# Prompt user for input
gameNameOne = input("Enter your summoner: ")


# Prompt user for input with a default value
tagLineOne = input("Enter tagline (default: NA1): ")

# Use default value if input is empty
tagLineOne = tagLineOne if tagLineOne.strip() else "NA1"

print(gameNameOne, tagLineOne)






# input username, region code here, asking for user input

# Prompt user for input of second player
gameNameTwo = input("Enter your friends summoner: ")


# Prompt user for input with a default value
tagLineTwo = input("Enter tagline (default: NA1): ")

# Use default value if input is empty
tagLineTwo = tagLineTwo if tagLineTwo.strip() else "NA1"

print(gameNameTwo, tagLineTwo)



hgillis NA1
lilonglee faker


# Get both players PUUIDs

In [229]:
# get player 1 puuid
puuidOne = get_puuid(gameName=gameNameOne, tagLine=tagLineOne)

# get player 2 puuid
puuidTwo = get_puuid(gameName=gameNameTwo, tagLine=tagLineTwo)

print(puuidOne)
print(puuidTwo)

V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoDgxwUtdIeVTZrmOzjeVHklBRlCJr1tz1g
7CKBN69b23h8oNhnZ_Q81daYeS0nPYj5aNo8lekjqnJ9WppSB9Z0gmZHGfr65-26KlKVn_gaZKGhcw


# Get both players match histories

In [230]:
# Get match IDs for the first game
match_idsOne = get_match_history(region='americas', puuid=get_puuid(gameName=gameNameOne, tagLine=tagLineOne))

# Initialize an empty DataFrame for the first game
matchHistoryDfOne = pd.DataFrame()

# Populate the DataFrame with match data for the first game
for match_id in match_idsOne:
    gameone = get_match_data_from_id(region='americas', matchId=match_id)
    matchDFOne = process_match_json(gameone, puuid=get_puuid(gameName=gameNameOne, tagLine=tagLineOne))
    matchHistoryDfOne = pd.concat([matchHistoryDfOne, matchDFOne])

# Get match IDs for the second game
match_idsTwo = get_match_history(region='americas', puuid=get_puuid(gameName=gameNameTwo, tagLine=tagLineTwo))

# Initialize an empty DataFrame for the second game
matchHistoryDfTwo = pd.DataFrame()

# Populate the DataFrame with match data for the second game
for match_id in match_idsTwo:
    gametwo = get_match_data_from_id(region='americas', matchId=match_id)
    matchDFTwo = process_match_json(gametwo, puuid=get_puuid(gameName=gameNameTwo, tagLine=tagLineTwo))
    matchHistoryDfTwo = pd.concat([matchHistoryDfTwo, matchDFTwo])

# Now, `matchHistoryDfOne` and `matchHistoryDfTwo` are isolated and contain data for different game names


In [231]:
matchHistoryDfOne


,match_id,game_mode,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,puuid,riot_id,riot_tag,summoner_id,summoner_name,win,champion,champion_transform,champion_level,kills,deaths,assists,summoner1_id,summoner2_id,gold_earned,total_minions_killed,total_neutral_minions_killed,early_surrender,surrender,first_blood,objectives_stolen,objectives_stolen_assists,total_damage_dealt_champions,total_damage_taken,total_damage_shielded_teammates,total_time_crowd_controlled,vision_score,wards_placed,item0,item1,item2,item3,item4,item5,item6,perk_keystone,perk_primary_row_1,perk_primary_row_2,perk_primary_row_3,perk_secondary_row_1,perk_secondary_row_2,perk_primary_style,perk_secondary_style,perk_shard_defense,perk_shard_flex,perk_shard_offense,damage_dealt_to_buildings,damage_dealt_to_objectives,damage_dealt_to_turrets,game_ended_in_surrender,individual_position,inhibitor_takedowns,largest_killing_spree,largest_multi_kill,total_damage_shielded_on_teammates,total_heals_on_teammates
0,NA1_5183290141,CLASSIC,[d66zRs2wr3In3cbz_fPo1cbUt99SyQNUr78FGsGcm_Bma...,1734323501921,2187,1734323746573,1734325933833,14.24.643.5128,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,True,26,0,16,1,4,29,7,4,10596,24,0,False,False,False,0,0,9960,32102,1835,181,45,21,3876,2065,3158,4005,3083,3113,3364,8214,8226,8210,8232,8126,8105,8200,8200,5001,5008,5007,2034,2135,2034,False,UTILITY,0,0,1,1835,2550
0,NA1_5183249627,CLASSIC,[d66zRs2wr3In3cbz_fPo1cbUt99SyQNUr78FGsGcm_Bma...,1734321321424,1858,1734321401751,1734323259816,14.24.643.5128,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,True,142,0,13,4,10,11,14,4,8870,10,0,False,True,False,0,0,17971,21762,0,135,39,17,3871,3020,6655,4628,3057,1052,3364,8128,8143,8138,8135,8233,8236,8100,8100,5001,5008,5008,1749,2599,1749,True,UTILITY,0,2,2,0,164
0,NA1_5181444806,CLASSIC,[zidX4wyMk3i2Kogfu0UrrxWhbwAgQ1nBHvCwdEi287nAS...,1734153465699,1733,1734153550843,1734155283741,14.24.643.5128,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,False,80,0,12,5,8,11,14,4,8986,20,0,False,False,False,0,0,13435,20347,0,39,54,21,3877,3142,3013,3179,3134,1037,3364,8005,9111,9105,8014,8126,8105,8000,8000,5001,5008,5008,258,3157,258,False,UTILITY,0,0,1,0,0
0,NA1_5181412050,CLASSIC,[d66zRs2wr3In3cbz_fPo1cbUt99SyQNUr78FGsGcm_Bma...,1734151584129,1476,1734151657800,1734153134085,14.24.643.5128,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,True,427,0,13,3,3,25,7,4,8573,8,0,False,False,False,0,0,7379,12907,9572,109,24,9,3870,3158,6617,3504,3113,4642,3364,8214,8226,8210,8237,8463,8453,8200,8200,5001,5008,5007,3937,4101,3937,False,UTILITY,2,2,1,9572,1343
0,NA1_5179458819,CLASSIC,[CBD1l26RzaFFa-cImBA7O9wwGvH3h7h3vxTCfqbY8UbgU...,1733964142972,3042,1733964228361,1733967270880,14.24.642.1879,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,True,27,0,18,3,18,23,6,4,12787,22,2,False,False,False,0,0,16730,62896,0,334,83,33,3876,2065,3116,3002,3013,1011,3364,8360,8306,8345,8410,5007,5010,8300,8300,5011,5010,5007,724,3594,724,False,UTILITY,0,0,1,0,0
0,NA1_5179431661,CLASSIC,[-34neXe-6Jv5gh-aG1oG30Qj4K2_nUyaQk7aKYAdmlP75...,1733962492140,1296,1733962606178,1733963902677,14.24.642.1879,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,False,99,0,10,2,7,4,14,4,5653,23,0,False,True,False,0,0,10801,11387,839,134,30,11,3871,3020,3118,1052,0,0,3364,8128,8126,8138,8106,5008,5001,8100,8100,5008,5001,5008,247,467,247,True,UTILITY,0,0,1,839,0
0,NA1_5176298198,CLASSIC,[-qcVfkj8M4tsiLxecUkLkTk8OtK7OBZwvihx3LOKlFuJY...,1733630947328,1593,1733630972745,1733632566345,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD..

In [232]:
matchHistoryDfTwo

,match_id,game_mode,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,puuid,riot_id,riot_tag,summoner_id,summoner_name,win,champion,champion_transform,champion_level,kills,deaths,assists,summoner1_id,summoner2_id,gold_earned,total_minions_killed,total_neutral_minions_killed,early_surrender,surrender,first_blood,objectives_stolen,objectives_stolen_assists,total_damage_dealt_champions,total_damage_taken,total_damage_shielded_teammates,total_time_crowd_controlled,vision_score,wards_placed,item0,item1,item2,item3,item4,item5,item6,perk_keystone,perk_primary_row_1,perk_primary_row_2,perk_primary_row_3,perk_secondary_row_1,perk_secondary_row_2,perk_primary_style,perk_secondary_style,perk_shard_defense,perk_shard_flex,perk_shard_offense,damage_dealt_to_buildings,damage_dealt_to_objectives,damage_dealt_to_turrets,game_ended_in_surrender,individual_position,inhibitor_takedowns,largest_killing_spree,largest_multi_kill,total_damage_shielded_on_teammates,total_heals_on_teammates
0,NA1_5183209013,ARAM,[7CKBN69b23h8oNhnZ_Q81daYeS0nPYj5aNo8lekjqnJ9W...,1734319005353,1567,1734319040164,1734320607783,14.24.643.5128,7CKBN69b23h8oNhnZ_Q81daYeS0nPYj5aNo8lekjqnJ9Wp...,lilonglee,faker,MzLb_kG3jqhJ1IBUphSm21_ESY84oLn4kQxOyxeVSSXfgrU,lolss,True,68,0,18,14,16,22,4,32,18019,53,0,False,False,False,0,0,61171,41601,0,254,0,0,2421,4645,3020,3089,3137,6653,2052,8229,8224,8210,8236,8138,8105,8200,8200,5001,5008,5008,905,905,905,False,Invalid,2,3,3,0,540
0,NA1_5183170531,ARAM,[7CKBN69b23h8oNhnZ_Q81daYeS0nPYj5aNo8lekjqnJ9W...,1734317220320,1562,1734317306223,1734318868260,14.24.643.5128,7CKBN69b23h8oNhnZ_Q81daYeS0nPYj5aNo8lekjqnJ9Wp...,lilonglee,faker,MzLb_kG3jqhJ1IBUphSm21_ESY84oLn4kQxOyxeVSSXfgrU,lolss,True,112,0,18,9,17,33,4,6,16561,56,0,False,False,True,0,0,44335,39492,0,306,0,0,3118,6653,4633,3135,3020,1058,2052,8229,8226,8210,8237,8014,8009,8200,8200,5001,5008,5008,886,886,886,False,Invalid,0,2,2,0,0
0,NA1_5183136676,ARAM,[b_ku8qamNpK0KO-zuyWbJZPdbQEKfFUtnv3MovKZmrGx8...,1734315538157,850,1734315685920,1734316536430,14.24.643.5128,7CKBN69b23h8oNhnZ_Q81daYeS0nPYj5aNo8lekjqnJ9Wp...,lilonglee,faker,MzLb_kG3jqhJ1IBUphSm21_ESY84oLn4kQxOyxeVSSXfgrU,lolss,True,104,0,16,9,9,26,4,32,11990,52,0,False,False,False,0,0,23064,20643,0,205,0,0,126697,3111,6676,3072,1036,0,2052,8021,9111,9104,8014,8139,8135,8000,8000,5001,5008,5005,2939,2939,2939,False,Invalid,1,5,3,0,0
0,NA1_5182506982,CLASSIC,[9HygRc1CjgC6_7dWepMEt2OUxE8A-oNtgEvMo8t_YFau-...,1734248833584,2124,1734248857781,1734250982520,14.24.643.5128,7CKBN69b23h8oNhnZ_Q81daYeS0nPYj5aNo8lekjqnJ9Wp...,lilonglee,faker,MzLb_kG3jqhJ1IBUphSm21_ESY84oLn4kQxOyxeVSSXfgrU,lolss,True,799,0,16,6,10,14,4,11,12553,31,141,False,False,False,0,0,27600,48458,0,340,22,4,3071,3161,1036,3047,6699,0,3364,8010,9111,9105,8014,8143,8106,8000,8000,5011,5008,5008,322,32645,322,False,JUNGLE,0,2,1,0,0
0,NA1_5182479806,CLASSIC,[7CKBN69b23h8oNhnZ_Q81daYeS0nPYj5aNo8lekjqnJ9W...,1734246604580,1894,1734246632196,1734248526038,14.24.643.5128,7CKBN69b23h8oNhnZ_Q81daYeS0nPYj5aNo8lekjqnJ9Wp...,lilonglee,faker,MzLb_kG3jqhJ1IBUphSm21_ESY84oLn4kQxOyxeVSSXfgrU,lolss,False,893,0,16,3,11,8,4,12,11409,211,0,False,True,False,0,0,27830,38528,0,256,14,5,3802,4629,1011,3020,1052,6653,3364,8437,8446,8473,8451,8017,8009,8400,8400,5011,5008,5005,4300,5917,4300,True,TOP,1,3,2,0,0
0,NA1_5182445564,CLASSIC,[pvVdpVQdAbNGN3QMu6HfmWPBloIP-uenc7_xPPiCXy8g8...,1734244268658,1868,1734244352125,1734246220207,14.24.643.5128,7CKBN69b23h8oNhnZ_Q81daYeS0nPYj5aNo8lekjqnJ9Wp...,lilonglee,faker,MzLb_kG3jqhJ1IBUphSm21_ESY84oLn4kQxOyxeVSSXfgrU,lolss,True,893,0,17,10,6,13,4,12,14077,218,5,False,False,False,0,0,30844,31818,0,335,17,7,3020,6653,4629,3118,3147,1082,3364,8112,8139,8138,8106,8226,8210,8100,8100,5001,5008,5005,5019,11390,5019,False,MIDDLE,1,6,2,0,0
0,NA1_5181935388,CLASSIC,[hA-qs9DlrQ-zAJ6jMBBp-O2F28eHuNUjzCszpt60jd7Ey...,1734212079168,1337,1734212110194,1734213447479,14.24.643.5128,7CKBN69b23h8oNhnZ_Q81daYeS0nPYj5aNo8lekjqnJ9Wp...,lilongle

In [233]:
pd.options.display.max_columns = 100

matchHistoryDfOne.replace(perk_dict).replace(perk_styles_dict)

# Replace only the 'champion' column with the values from champ_dict
matchHistoryDfOne['champion'] = matchHistoryDfOne['champion'].replace(champ_dict)

# replace the item columns with the the actual items names from the item_dict
matchHistoryDfOne['item0'] = matchHistoryDfOne['item0'].replace(item_dict)
matchHistoryDfOne['item1'] = matchHistoryDfOne['item1'].replace(item_dict)
matchHistoryDfOne['item2'] = matchHistoryDfOne['item2'].replace(item_dict)
matchHistoryDfOne['item3'] = matchHistoryDfOne['item3'].replace(item_dict)
matchHistoryDfOne['item4'] = matchHistoryDfOne['item4'].replace(item_dict)
matchHistoryDfOne['item5'] = matchHistoryDfOne['item5'].replace(item_dict)
matchHistoryDfOne['item6'] = matchHistoryDfOne['item6'].replace(item_dict)

# Replace the values based on the dictionaries and drop the first 12 columns
filtered_DfOne = matchHistoryDfOne.replace(perk_dict).replace(perk_styles_dict)

#drop columns i dont want
filtered_DfOne = filtered_DfOne.drop(columns = ['champion_transform', 'total_neutral_minions_killed', 'early_surrender', 'surrender',  'first_blood', 'objectives_stolen',  
                                                'objectives_stolen_assists', 'champion_level', 'summoner1_id', 'summoner2_id', 'item0', 'item1', 'item2', 'item3', 'item4', 
                                                'item5', 'item6', 'perk_keystone', 'perk_primary_row_1', 'perk_primary_row_2', 'perk_primary_row_3', 'perk_secondary_row_1', 
                                                'perk_secondary_row_2', 'perk_primary_style', 'perk_secondary_style', 'perk_shard_defense', 'perk_shard_flex', 'perk_shard_offense',
                                                'match_id', 'participants', 'game_creation', 'game_duration', 'game_start_timestamp', 'game_end_timestamp', 'puuid', 'riot_id', 'riot_tag', 'summoner_id', 'summoner_name', 'patch'

])

# drop columns where game_mode isn't CLASSIC
filtered_DfOne = filtered_DfOne[filtered_DfOne['game_mode'] == 'CLASSIC'].reset_index(drop=True)

# Replace True with 'Win' and False with 'Loss'
filtered_DfOne['win'] = filtered_DfOne['win'].replace({True: 'Win', False: 'Loss'})




# Output the result to a CSV file
filtered_DfOne.to_csv('match_hist1.csv', index=False)

filtered_DfOne



,game_mode,win,champion,kills,deaths,assists,gold_earned,total_minions_killed,total_damage_dealt_champions,total_damage_taken,total_damage_shielded_teammates,total_time_crowd_controlled,vision_score,wards_placed,damage_dealt_to_buildings,damage_dealt_to_objectives,damage_dealt_to_turrets,game_ended_in_surrender,individual_position,inhibitor_takedowns,largest_killing_spree,largest_multi_kill,total_damage_shielded_on_teammates,total_heals_on_teammates
0,CLASSIC,Win,Zilean,1,4,29,10596,24,9960,32102,1835,181,45,21,2034,2135,2034,False,UTILITY,0,0,1,1835,2550
1,CLASSIC,Win,Zoe,4,10,11,8870,10,17971,21762,0,135,39,17,1749,2599,1749,True,UTILITY,0,2,2,0,164
2,CLASSIC,Loss,Pantheon,5,8,11,8986,20,13435,20347,0,39,54,21,258,3157,258,False,UTILITY,0,0,1,0,0
3,CLASSIC,Win,Ivern,3,3,25,8573,8,7379,12907,9572,109,24,9,3937,4101,3937,False,UTILITY,2,2,1,9572,1343
4,CLASSIC,Win,Singed,3,18,23,12787,22,16730,62896,0,334,83,33,724,3594,724,False,UTILITY,0,0,1,0,0
5,CLASSIC,Loss,Lux,2,7,4,5653,23,10801,11387,839,134,30,11,247,467,247,True,UTILITY,0,0,1,839,0
6,CLASSIC,Win,Graves,3,9,10,9643,11,13478,23269,0,409,14,6,182,10975,182,False,JUNGLE,0,0,1,0,0
7,CLASSIC,Win,Nocturne,9,4,11,12461,23,22108,29910,0,607,18,8,5686,23557,5686,False,JUNGLE,1,3,2,0,0
8,CLASSIC,Win,Graves,13,11,9,15917,11,28324,33523,0,628,11,6,2696,40154,2696,False,JUNGLE,2,4,2,0,0
9,CLASSIC,Win,Ivern,1,8,15,13329,5,11034,29185,12208,168,23,11,4461,46335,4461,False,JUNGLE,1,0,1,12208,1365


In [234]:
pd.options.display.max_columns = 100

matchHistoryDfTwo.replace(perk_dict).replace(perk_styles_dict)

# Replace only the 'champion' column with the values from champ_dict
matchHistoryDfTwo['champion'] = matchHistoryDfTwo['champion'].replace(champ_dict)

# replace the item columns with the the actual items names from the item_dict
matchHistoryDfTwo['item0'] = matchHistoryDfTwo['item0'].replace(item_dict)
matchHistoryDfTwo['item1'] = matchHistoryDfTwo['item1'].replace(item_dict)
matchHistoryDfTwo['item2'] = matchHistoryDfTwo['item2'].replace(item_dict)
matchHistoryDfTwo['item3'] = matchHistoryDfTwo['item3'].replace(item_dict)
matchHistoryDfTwo['item4'] = matchHistoryDfTwo['item4'].replace(item_dict)
matchHistoryDfTwo['item5'] = matchHistoryDfTwo['item5'].replace(item_dict)
matchHistoryDfTwo['item6'] = matchHistoryDfTwo['item6'].replace(item_dict)

# Replace the values based on the dictionaries and drop the first 12 columns
filtered_DfTwo = matchHistoryDfTwo.replace(perk_dict).replace(perk_styles_dict)

#drop columns i dont want
filtered_DfTwo = filtered_DfTwo.drop(columns = ['champion_transform', 'total_neutral_minions_killed', 'early_surrender', 'surrender',  'first_blood', 'objectives_stolen',  
                                                'objectives_stolen_assists', 'champion_level', 'summoner1_id', 'summoner2_id', 'item0', 'item1', 'item2', 'item3', 'item4', 
                                                'item5', 'item6', 'perk_keystone', 'perk_primary_row_1', 'perk_primary_row_2', 'perk_primary_row_3', 'perk_secondary_row_1', 
                                                'perk_secondary_row_2', 'perk_primary_style', 'perk_secondary_style', 'perk_shard_defense', 'perk_shard_flex', 'perk_shard_offense',
                                                'match_id', 'participants', 'game_creation', 'game_duration', 'game_start_timestamp', 'game_end_timestamp', 'puuid', 'riot_id', 'riot_tag', 'summoner_id', 'summoner_name', 'patch'
])

# drop columns where game_mode isn't CLASSIC
filtered_DfTwo = filtered_DfTwo[filtered_DfTwo['game_mode'] == 'CLASSIC'].reset_index(drop=True)

# Replace True with 'Win' and False with 'Loss'
filtered_DfTwo['win'] = filtered_DfTwo['win'].replace({True: 'Win', False: 'Loss'})

# Output the result to a CSV file
filtered_DfTwo.to_csv('match_hist1.csv', index=False)

filtered_DfTwo


,game_mode,win,champion,kills,deaths,assists,gold_earned,total_minions_killed,total_damage_dealt_champions,total_damage_taken,total_damage_shielded_teammates,total_time_crowd_controlled,vision_score,wards_placed,damage_dealt_to_buildings,damage_dealt_to_objectives,damage_dealt_to_turrets,game_ended_in_surrender,individual_position,inhibitor_takedowns,largest_killing_spree,largest_multi_kill,total_damage_shielded_on_teammates,total_heals_on_teammates
0,CLASSIC,Win,Ambessa,6,10,14,12553,31,27600,48458,0,340,22,4,322,32645,322,False,JUNGLE,0,2,1,0,0
1,CLASSIC,Loss,Aurora,3,11,8,11409,211,27830,38528,0,256,14,5,4300,5917,4300,True,TOP,1,3,2,0,0
2,CLASSIC,Win,Aurora,10,6,13,14077,218,30844,31818,0,335,17,7,5019,11390,5019,False,MIDDLE,1,6,2,0,0
3,CLASSIC,Loss,Volibear,0,8,0,6869,139,11271,25818,0,246,12,7,3808,3808,3808,False,TOP,0,0,0,0,0
4,CLASSIC,Win,Irelia,6,6,5,13693,234,33042,31811,0,141,16,9,9727,15873,9727,False,TOP,2,4,2,0,0
5,CLASSIC,Loss,Aurora,3,5,6,10684,210,23507,23068,0,273,13,6,6894,12705,6894,False,MIDDLE,0,2,1,0,0
6,CLASSIC,Win,Viktor,8,3,14,12270,177,34177,17627,0,295,18,10,4405,5325,4405,False,MIDDLE,2,5,1,0,0
7,CLASSIC,Win,Jayce,15,9,12,22275,310,72158,44035,0,325,30,16,8894,29769,8894,False,MIDDLE,2,6,2,0,0
8,CLASSIC,Loss,Viktor,3,5,3,8858,157,17707,11048,0,72,14,7,5911,13014,5911,True,MIDDLE,0,2,1,0,0
9,CLASSIC,Loss,Jax,6,11,5,12905,194,34381,46339,0,159,30,13,2716,6908,2716,True,TOP,0,2,1,0,0
